# Forecast with ARCH / GARCH / EGARCH
In this notebook we take all the results from the bruteforce and we forecast with the models

We then gather the squared errors for each model. In order to compute the MCS on them.

In [12]:
from arch import arch_model
import datetime as dt
from libraries.Cryptov2 import Cryptocurrency
import pandas as pd
import numpy as np
import datetime
import arch
import matplotlib.pyplot as plt
from arch.univariate import arch_model
from arch.univariate import ARCH, GARCH, EGARCH, MIDASHyperbolic
from arch.univariate import ConstantMean, ZeroMean
from arch.univariate import Normal, SkewStudent, StudentsT
from sklearn.preprocessing import StandardScaler

In [13]:
BTC_models = pd.read_csv("./models/ts/fits/BTC-results.csv", index_col=0)
EOS_models = pd.read_csv("./models/ts/fits/EOS-results.csv", index_col=0)
ETH_models = pd.read_csv("./models/ts/fits/ETH-results.csv", index_col=0)
IOTA_models = pd.read_csv("./models/ts/fits/IOTA-results.csv", index_col=0)

crypto_model_list = [BTC_models,IOTA_models,ETH_models,EOS_models]

In [14]:
BTC = Cryptocurrency('BTC', data=None ,url = 'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end=20191001')
IOTA = Cryptocurrency('IOTA', data=None ,url = 'https://coinmarketcap.com/currencies/iota/historical-data/?start=20130428&end=20191001')
ETH = Cryptocurrency('ETH', data=None ,url = 'https://coinmarketcap.com/currencies/ethereum/historical-data/?start=20130428&end=20191001')
EOS = Cryptocurrency('EOS', data=None ,url = 'https://coinmarketcap.com/currencies/eos/historical-data/?start=20130428&end=20191001')

Cryptocurrency V.0.1
Data for BTC already exists.
Reading...
Done! 2348 rows and 49 columns
Cryptocurrency V.0.1
Data for IOTA already exists.
Reading...
Done! 841 rows and 49 columns
Cryptocurrency V.0.1
Data for ETH already exists.
Reading...
Done! 1517 rows and 49 columns
Cryptocurrency V.0.1
Data for EOS already exists.
Reading...
Done! 823 rows and 49 columns


In [15]:
crypto_list = [BTC,IOTA,ETH,EOS]

In [16]:
for each in crypto_list:
    each.data['log_price'] = np.log(each.data['close'])
    each.data['pct_change'] = each.data['log_price'].diff()
    each.data['stdev14'] = each.data['pct_change'].rolling(window=14, center=False).std(ddof=0)
    each.data['stdev30'] = each.data['pct_change'].rolling(window=30, center=False).std(ddof=0)
    each.data['stdev60'] = each.data['pct_change'].rolling(window=60, center=False).std(ddof=0)

    each.data['hvol14'] = each.data['stdev14'] * (365**0.5) # Annualize.
    each.data['hvol30'] = each.data['stdev30'] * (365**0.5) # Annualize.
    each.data['hvol60'] = each.data['stdev60'] * (365**0.5) # Annualize.

    each.data['variance14'] = each.data['hvol14']**2
    each.data['variance30'] = each.data['hvol30']**2
    each.data['variance60'] = each.data['hvol60']**2

    each.data = each.data.dropna() # Remove rows with blank cells.
    each.data['returns'] = each.data['pct_change'] * 100

In [17]:
def compute_mse_and_volatility(vol_model,dist,crypto,p,q):
    import numpy as np
    import matplotlib.pyplot as plt
    
    n_days = len(crypto.data.index) * 5 // 6
    last_date = dt.datetime(crypto.data.index[n_days].year,crypto.data.index[n_days].month,crypto.data.index[n_days].day)
    start_date = dt.datetime(crypto.data.index[0].year,crypto.data.index[0].month,crypto.data.index[0].day)
    
    returns = crypto.data['returns']

    am = ConstantMean(returns)
    
    # Our distribution
    if dist == 'normal':
        am.distribution = Normal()
    elif dist == 'studentst':
        am.distribution = StudentsT()
    elif dist == 'skewstudent':
        am.distribution = SkewStudent()

    # Our volatility process
    ## Double  == is super important is doesnt work
    if vol_model == "ARCH":
        am.volatility = ARCH(p=p)
    elif vol_model == "GARCH":
        am.volatility = GARCH(p=p, q=q)
    elif vol_model == "EGARCH":
        am.volatility = EGARCH(p=p, q=q)
    
    # We plot the series 
    x2 = crypto.data['hvol14']

    res = am.fit(disp="off", last_obs=last_date)
    
    forecasts = res.forecast(horizon=1, start=start_date, method='analytic')
    
    x1 = np.sqrt(0.01*forecasts.variance)
    x1 = x1['h.1']
    
    squared_error = (x1-x2)**2
    
    return squared_error[last_date:]
    

In [18]:
for crypto in crypto_list:
    df = pd.read_csv(f"./models/ts/fits/{crypto.name}-results.csv", index_col=0)
    df_squared_error = pd.DataFrame()
    for idx, row in df.iterrows():
        squared_returns = compute_mse_and_volatility(row['volatility_model'],row['dist'],crypto,row['p'],row['q'])
        df_squared_error[f'{crypto.name}-model-{idx}'] = squared_returns
        
    df_squared_error.to_csv(f"./mcs/data/models/TS/{crypto.name}/MCS/{crypto.name}-ts-models.csv")